<a href="https://colab.research.google.com/github/neelkantnewra/QAGeneration/blob/main/QA_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install -U transformers==3.0.0
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git 
!pip install pikepdf
!pip install tika

     |████████████████████████████████| 77 kB 4.7 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=8fc2bdefac12c8332055fdcbb4f535ec57ee22aa5d4f8c6ea05831d42928725b
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 754 kB 7.1 MB/s 
     |████████████████████████████████| 3.0 MB 44.1 MB/s 
     |████████████████████████████████| 895 kB 84.0 MB/s 
     |████████████████████████████████| 1.2 MB 59.5 MB/s 
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: C

In [2]:
import PyPDF2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import nltk
import pikepdf
from tika import parser

In [4]:
def create_pdf_copy(pdf_file_name):
  with pikepdf.open(pdf_file_name,allow_overwriting_input=True) as pdf:
    new_filename = f'{pdf_file_name}'
    pdf.save(new_filename)
    return  new_filename

def extract_pdf_content(pdf_file_name):
  # plain text output
  # parsedPDF = parser.from_file("restrictive_editing_removed.pdf")

  # XHTML output
  parsedPDF = parser.from_file(pdf_file_name, xmlContent=False)

  pdf = parsedPDF["content"]
  pdf = pdf.replace('\n\n', '\n')
  return pdf

def password_required(pdf_file_name):
  try:
    pikepdf.open(pdf_file_name)

  except pikepdf.PasswordError as error:
    return ('password required')

  except pikepdf.PdfError as results:
    return ('cannot open file')



filename = '/content/SodaPDF-converted-tc.pdf'
password = password_required(filename)
if password != None:
  print (password)
elif password == None:
  pdf_file = create_pdf_copy(filename)
  results = extract_pdf_content(pdf_file)

2021-08-25 10:32:45,473 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-08-25 10:32:47,888 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-08-25 10:32:48,316 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [5]:
%cd question_generation
from pipelines import pipeline
content = results
c = content.splitlines()
nlp = pipeline("question-generation")


[Errno 2] No such file or directory: 'question_generation'
/content/question_generation


Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [6]:
content = c
superfinal = []
  
new_c = []

for text in content:
  new_c.append(text.strip())

# Removing the empty text

while "" in new_c:
  new_c.remove("")


# Applying the model and reteriving question answer in ans list
ans = []
for text in new_c:
  try:
    ans.append(nlp(text))
  except (ValueError,AssertionError):
    pass

# Since we got repeated Answer:question pair we removed it using set
finalans = []
for i in range(len(ans)):
  l = ans[i]
  finalans.append([dict(t) for t in {tuple(d.items()) for d in l}])
superfinal.append(finalans)

In [7]:
flat = [item for sublist in superfinal for item in sublist] # open List of List of List
flat_list = [item for sublist in flat for item in sublist] # open List of List


In [10]:
# dumping the file in json format 

from google.colab import files
import json

x = json.dumps (flat_list)
y = x.encode('utf-8')
with open('SodaPDF-converted.json', 'w') as f:
  f.write(str(y))

files.download('SodaPDF-converted.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>